In [19]:
# Import dependencies and libraries
import gmaps
import pandas as pd
import os

# Google developer API key
from config import api_key

# Configure gmaps
gmaps.configure(api_key=api_key)


In [55]:
# Importing csv with cities data

## Retrieving file list
avail_files = []
path = os.path.join('..', 'WeatherPy', 'results_csv')
for _ in os.scandir(path):
    if os.path.isfile:
        (avail_files.append(_.name))
    
# Saving the files list to a dataframe to allow user input by number
avail_files = pd.DataFrame({'Files Available': avail_files})
print (avail_files)

# For testing purposes, switch comment after testing
# import_file = int (input("Type the index of the file you want to use: "))
import_file = 0

# Opening csv file and converting to dataframe
csv_path = os.path.join('..', 'WeatherPy', 'results_csv', avail_files.loc[import_file, 'Files Available'])
weather_data = pd.read_csv(csv_path)

weather_data

               Files Available
0  weather_data_1619218389.csv


,Unnamed: 0,Cities,Latitude,Longitude,Temperature (°F),Humidity,Cloudiness,Wind Speed (mph)
0,0,sorland,67.6670,12.6934,36.54,59,100,21.39
1,1,tromso,69.6496,18.9570,28.18,93,40,3.44
2,2,dikson,73.5069,80.5464,32.20,97,100,18.57
3,3,klaksvik,62.2266,-6.5890,42.80,100,40,5.75
4,4,clyde river,70.4692,-68.5914,17.60,100,90,5.75
...,...,...,...,...,...,...,...,...
637,637,talcahuano,-36.7167,-73.1167,57.20,94,40,11.50
638,638,luderitz,-26.6481,15.1594,63.27,80,44,8.28
639,639,paso de carrasco,-34.8603,-56.0522,70.65,60,0,7.54
640,640,teno,-34.8667,-71.1833,59.49,67,90,6.02


In [39]:
print (import_file)


0
